In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp -r "/content/drive/My Drive/scRNA" /content/
%cd /content/scRNA
!ls

/content/scRNA
build  COLAB_SETUP.md  config  dataset	h5reader  include  logs  source


Install Dependencies

In [3]:
   !apt-get update
   !apt-get install -y libhdf5-dev pkg-config

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,153 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,835 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,539 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-b

Build project

In [4]:
import os
os.chdir('/content/scRNA')
!pwd

/content/scRNA


In [6]:
import os
os.chdir('/content/scRNA')

# Create build directory
!mkdir -p build

# Step 1: Compile CUDA file with -DUSE_CUDA (CRITICAL!)
print("Step 1: Compiling CUDA file...")
!nvcc -c -std=c++17 -O3 -DUSE_CUDA -Iinclude -I/usr/local/cuda/include -x cu source/dense_spmm_cuda.cu -o build/dense_spmm_cuda.o

# Step 2: Build executable
print("\nStep 2: Building executable...")
!g++ -std=c++17 -O3 -Wall -fopenmp -DUSE_CUDA -Iinclude \
    source/test_run5_cuda_tiled_spmm.cpp \
    source/permutation.cpp \
    source/disk_to_memory.cpp \
    source/spmm_baseline.cpp \
    source/tiler.cpp \
    source/tile_spmm.cpp \
    build/dense_spmm_cuda.o \
    -o build/run5 \
    -I/usr/include/hdf5/serial \
    -L/usr/lib/x86_64-linux-gnu/hdf5/serial \
    -lhdf5 -lhdf5_cpp \
    -I/usr/local/cuda/include -L/usr/local/cuda/lib64 -lcudart -lcublas

# Step 3: Verify
print("\nStep 3: Verifying build...")
!ls -lh build/run5

Step 1: Compiling CUDA file...

Step 2: Building executable...

Step 3: Verifying build...
-rwxr-xr-x 1 root root 151K Nov 27 23:15 build/run5


In [9]:
import os

os.chdir('/content/scRNA')

# Create build directory
!mkdir -p build

# Step 1: Compile CUDA file with -DUSE_CUDA
!nvcc -c -std=c++17 -O3 -DUSE_CUDA -Iinclude -I/usr/local/cuda/include -x cu source/dense_spmm_cuda.cu -o build/dense_spmm_cuda.o

# Step 2: Build executable with all source files
!g++ -std=c++17 -O3 -Wall -fopenmp -DUSE_CUDA -Iinclude \
    source/test_run5_cuda_tiled_spmm.cpp \
    source/permutation.cpp \
    source/disk_to_memory.cpp \
    source/spmm_baseline.cpp \
    source/tiler.cpp \
    source/tile_spmm.cpp \
    build/dense_spmm_cuda.o \
    -o build/run5 \
    -I/usr/include/hdf5/serial \
    -L/usr/lib/x86_64-linux-gnu/hdf5/serial \
    -lhdf5 -lhdf5_cpp \
    -I/usr/local/cuda/include -L/usr/local/cuda/lib64 -lcudart -lcublas

# Verify
!ls -lh build/run5

-rwxr-xr-x 1 root root 151K Nov 27 23:45 build/run5


In [13]:
import os
os.chdir('/content/scRNA/build')
!./run5 d4.h5 w4.h5

=== CUDA Tiled SpMM Test (run5) ===
CUDA: ENABLED

[disk_to_memory] Loading X from: ../dataset/X/d4.h5
[disk_to_memory] X shape: 134920 x 2711, nnz: 24511186
[disk_to_memory] Successfully loaded and transposed X to CSR
[disk_to_memory] Loading W from: ../dataset/W/w4.h5
[disk_to_memory] W shape: 2711 x 32
tiles: 90687
dense: 62326
sparse: 28361
Saved result to: ../dataset/Y/y4_cuda.h5
✗ Y mismatches vs Y_check: 37 elements
spmm done
